Computing Point-in-Time Residual Returns
In this homework, we will use regressions to compute beta-adjusted "residual" returns in a point-in-time fashion suitable for backtesting / live trading.


1. Download Daily Bars for FB, AAPL, AMZN, NFLX, GOOGL and QQQ from yahoo finance starting 2016-01-01. Use the Adj Close to compute daily returns.
2. Now, let's compute the beta of FB, AAPL, AMZN, NFLX, GOOGL using QQQ as our benchmark. You can think of this as the beta these stocks have to their industry (tech). In practice,  we have to use some lookback window to compute the beta. Let's use 252 (1 year, excluding wknds/holidays). So, for each day, the betas should be computed using the most recent 252 data points.
3. Using the betas, compute an "alpha" on each day. This is also known as a "residual return".
4. Compare the volatility of the residual returns to that of the original returns. What do you notice?
5. Compare the pairwise correlations of the residual returns to that of the original returns. What do you notice?
6. Compute the information ratio for each of these stocks and compare that to the sharpe ratio.


In [59]:
import yfinance as yf
import pandas as pd
import numpy as np

univ = ['META', 'AAPL', 'AMZN', 'NFLX', 'GOOGL', 'QQQ']
daily_ret = yf.download(univ, start = '2016-01-01')['Adj Close'].pct_change()
daily_ret

[*********************100%***********************]  6 of 6 completed


Ticker,AAPL,AMZN,GOOGL,META,NFLX,QQQ
Date,,,,,,
2016-01-04 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN
2016-01-05 00:00:00+00:00,-0.025059,-0.005024,0.002752,0.004989,-0.020917,-0.001735
2016-01-06 00:00:00+00:00,-0.019570,-0.001799,-0.002889,0.002336,0.093071,-0.009606
2016-01-07 00:00:00+00:00,-0.042205,-0.039058,-0.024140,-0.049043,-0.026513,-0.031313
2016-01-08 00:00:00+00:00,0.005288,-0.001464,-0.013617,-0.006025,-0.027671,-0.008201
...,...,...,...,...,...,...
2024-12-17 00:00:00+00:00,0.009720,-0.007642,-0.006305,-0.007689,-0.002117,-0.004404
2024-12-18 00:00:00+00:00,-0.021422,-0.045987,-0.035923,-0.035920,-0.032183,-0.036077
2024-12-19 00:00:00+00:00,0.007015,0.012561,0.000584,-0.002713,0.014041,-0.004453


In [60]:
import statsmodels.api as sm
from statsmodels.regression.rolling import RollingOLS

vol = daily_ret.rolling(252).std()
corr = daily_ret.rolling(252).corr(daily_ret['QQQ'])
beta = corr.mul(vol).div(vol['QQQ'], axis=0)
beta

Ticker,AAPL,AMZN,GOOGL,META,NFLX,QQQ
Date,,,,,,
2016-01-04 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN
2016-01-05 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN
2016-01-06 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN
2016-01-07 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN
2016-01-08 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
2024-12-17 00:00:00+00:00,0.702577,1.112893,0.890127,1.183829,0.894814,1.0
2024-12-18 00:00:00+00:00,0.700504,1.117038,0.892349,1.174102,0.893031,1.0
2024-12-19 00:00:00+00:00,0.698647,1.115149,0.891518,1.172671,0.888681,1.0


In [61]:
alpha = daily_ret-(beta.multiply(daily_ret['QQQ'], axis=0))
alpha

Ticker,AAPL,AMZN,GOOGL,META,NFLX,QQQ
Date,,,,,,
2016-01-04 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN
2016-01-05 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN
2016-01-06 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN
2016-01-07 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN
2016-01-08 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
2024-12-17 00:00:00+00:00,0.012814,-0.002741,-0.002385,-0.002476,0.001823,-1.474515e-17
2024-12-18 00:00:00+00:00,0.003850,-0.005688,-0.003729,0.006438,0.000035,-9.714451e-17
2024-12-19 00:00:00+00:00,0.010126,0.017527,0.004554,0.002510,0.017998,-1.474515e-17


In [62]:
vol = {}
vol['original'] = daily_ret.std()*np.sqrt(252)
vol['residual'] = alpha.std()*np.sqrt(252)
vol = pd.DataFrame(vol).drop('QQQ')
vol

,original,residual
Ticker,,
AAPL,0.286691,0.161733
AMZN,0.326435,0.202410
GOOGL,0.284400,0.173760
META,0.386116,0.278643
NFLX,0.427831,0.332924


In [63]:
daily_ret.corr()

Ticker,AAPL,AMZN,GOOGL,META,NFLX,QQQ
Ticker,,,,,,
AAPL,1.000000,0.575623,0.626404,0.526883,0.442287,0.818243
AMZN,0.575623,1.000000,0.650439,0.602146,0.538379,0.767362
GOOGL,0.626404,0.650439,1.000000,0.635641,0.465538,0.802448
META,0.526883,0.602146,0.635641,1.000000,0.466983,0.698086
NFLX,0.442287,0.538379,0.465538,0.466983,1.000000,0.595464
QQQ,0.818243,0.767362,0.802448,0.698086,0.595464,1.000000


In [64]:
alpha.corr()

Ticker,AAPL,AMZN,GOOGL,META,NFLX,QQQ
Ticker,,,,,,
AAPL,1.000000,-0.119902,-0.085414,-0.090384,-0.102679,-0.034368
AMZN,-0.119902,1.000000,0.065048,0.111035,0.148133,0.025035
GOOGL,-0.085414,0.065048,1.000000,0.162404,-0.048981,-0.007075
META,-0.090384,0.111035,0.162404,1.000000,0.090561,0.003880
NFLX,-0.102679,0.148133,-0.048981,0.090561,1.000000,-0.006253
QQQ,-0.034368,0.025035,-0.007075,0.003880,-0.006253,1.000000


In [65]:
ratios = {}
ratios['IR'] = alpha.mean()/alpha.std()*np.sqrt(252)
ratios['SR'] = daily_ret.mean()/daily_ret.std()*np.sqrt(252)
ratios = pd.DataFrame(ratios).drop('QQQ')
ratios

,IR,SR
Ticker,,
AAPL,0.541864,1.065051
AMZN,0.131144,0.833365
GOOGL,0.068287,0.785125
META,0.095680,0.707771
NFLX,0.276892,0.771354
